# Cahier des charges

| Fonction | Récupérer les données des stations Vélo'V |
| --- | --- |
| Objectif | Obtenir des données exploitables sur les stations Vélo'V installées et fonctionnelles, et le nombre de vélos et de places libres en temps réel |
| Contraintes | - Utiliser l'API Vélo'V <br>- Utiliser les packages *requests* et *pandas* <br>- Obtenir un tableau *pandas* en sortie |

| Fonction | Exporter des objets exploitables dans un SIG |
| --- | --- |
| Objectif | Obtenir des données vectorielles ponctuelles sur les stations Vélo'V en temps réel |
| Contraintes | - Export au format *shapefile* ou *geopackage* <br>- Utiliser le package *geopandas* <br>- Exporter dans un système de coordonnées projetées |

## Couche de points attendue 
![Couche de points attendue](img/stations.png "Couche de points attendue")

# Algorithmie

L'algorithmie peut se faire, à votre préférence, sur une feuille de papier en dessinant un [organigramme de programmation](https://fr.wikipedia.org/wiki/Organigramme_de_programmation), ou en rédigant du [pseudo-code](https://fr.wikipedia.org/wiki/Pseudo-code). Vous trouverez ci-dessous un exemple d'organigramme pour la première étape, et un exemple de pseudo-code pour la seconde. Vous pouvez cependant rédiger un seul algorithme pour toutes les étapes de cet objectif.

Voici quelques grandes étapes pour vous aider à réaliser cet objectif. Ces étapes ne sont pas obligatoires, vous pouvez décider de procéder complètement autrement si vous le souhaitez.

### Exemple d'organigramme de programmation pour cette étape
![Algorithme API VeloV](img/Algo_API_VeloV.png "Algorithme API VeloV")

## 1. Récupérer des données via l'API

Il faut d'abord rechercher les points de terminaison de l'API qui nous intéresse. Un point de terminaison se présente généralement sous la forme d'une URL à appeler avec des paramètres. L'URL renvoie un JSON contenant les données structurées. 

Recherchez l'URL permettant de récolter les statuts des stations Vélo'V, ainsi que l'URL qui permet de récupérer toutes les informations de localisation de chaque station.

## 2. Stocker cette réponse dans un DataFrame *pandas*

Pour faciliter le traitement de données tabulaires, l'utilisation d'un DataFrame *pandas* est conseillée. Il faudra bien veiller à mettre au propre les différents champs et leurs types.

### Exemple de pseudo-code pour cette étape

```
ENTRÉE: reponse_api_status
ENTRÉE: reponse_api_information
SORTIE: tableau_pandas

FAIRE donnees_velov = extraire json de la reponse_api_status
FAIRE infos_stations = extraire json de la reponse_api_information
SI (nb de lignes donnees_velov >= 1 ET nb de lignes infos_stations >= 1)
    FAIRE donnees_velov_completes = jointure attributaire de donnees_velov et infos_stations
    RENVOYER donnees_velov_completes
SINON
    RENVOYER ERREUR
```

## 3. Convertir le DataFrame *pandas* en GeoDataFrame *geopandas*

Il faut d'abord rechercher dans les données de l'API les champs qui nous permettrons de transformer de simples lignes d'un tableau en objets géographiques. Pour transformer le DataFrame en GeoDataFrame, il faudra renseigner également le type de géométrie à associer à chaque ligne et comment créer cette géométrie. Avant l'export, penser à reprojeter les données dasn un système de coordonnées projetées.

# Programmation

Ci-dessous, vous trouverez des éléments qu ipeuvent vous aider à programmer votre algorithme.

## 1. Récupération des données via l'API

Pour requêter cette URL et récupérer le JSON directement dans Python, vous pouvez utiliser le package *requests*.

In [ ]:
# On importe tout le package requests
import requests

# On récupère les données
reponse = requests.get(url='https://download.data.grandlyon.com/files/rdata/jcd_jcdecaux.jcdvelov/station_status.json')

# La fonction get du package requests va renvoyer un objet de type Reponse
print(reponse)

On peux vérifier le code HTTP de la réponse, puis ce quelle contient sous forme d'un JSON.

In [ ]:
print(reponse.status_code)

In [ ]:
print(reponse.json())

In [ ]:
# On stocke dans une variable la partie de la réponse qui nous intéresse
reponse_api = reponse.json()['data']['stations']

Refaire une requête pour interroger cette fois-ci l'URL de l'API *station_information.json*.

## 2. Stocker cette réponse dans un DataFrame *pandas*

Pour faciliter le traitement de données tabulaires, l'utilisation d'un DataFrame *pandas* est conseillée. Si nos données sont déjà bien structurées, la plupart du temps, *pandas* est capable de les interpréter directement de cette manière : 

In [ ]:
import pandas as pd
donnees_velov = pd.DataFrame(data=reponse_api)
donnees_velov

Si cela ne fonctionne pas, il faudra éplucher la [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) sur la manière de créer un DataFrame à partir de nos données. 

A ce stade, il est conseillé d'explorer un peu les données pour s'assurer qu'elles sont correctement récupérées. On peux, par exemple, faire un boxplot pour le nombre de vélos actuellement disponibles par station comme ci-desous.

In [ ]:
graph_chronique = donnees_velov.boxplot(column=['num_bikes_available', 'num_docks_available'], rot=45)

Certaines colones sont des booléens (valeurs vraies ou fausses). Il faudra faire une boucle sur les trois colonnes, et utiliser la méthode `.astype(bool)` pour transformer les valeurs dans le bon format. Vérifiez le résultat en affichant les informations du DataFrame avec la méthode `.info()`, puis en affichant le DataFrame.

Vous pouvez filtrer les stations installées et fonctonnelles dès à présent.

Par ailleurs, une colonne `num_docks_total` (somme de `num_bikes_available` et de `num_docks_available`) pourrait s'avérer utile. Vous pouvez la calculer à ce stade.

## 3. Convertir le DataFrame *pandas* en GeoDataFrame *geopandas*
Dans notre tableau, les stations sont identifiées par leur id. Pour savoir à quoi correspondent ces id, il faut récupérer des données sur un autre le point de terminaison station_information.json.

In [ ]:
reponse = requests.get('<URL>')

infos_stations = pd.DataFrame(reponse.json()['data']['stations'])

On réalise ensuite une jointure attributaire avec la fonction `merge()` du module principal de pandas.

Pour transformer le résultat en GeoDataFrame, il faut ensuite créer des objets vectoriels de type point à partir des données latitude et longitude du tableau. On utilisera pour cela [geopandas.points_from_xy()](https://geopandas.org/en/stable/docs/reference/api/geopandas.points_from_xy.html) pour définir les géométries au moment de la conversion du DataFrame.

Une fois le GeoDataFrame créé, on fait en sorte de passer d'un système longitude/latitude à un système de coordonnées projetées de votre choix. Le package *geopandas* dispose de tous les outils pour réaliser cela simplement.

Une fois le ménage dans les champs terminé, on peux enregistrer notre tableau de données spatiales sur le disque (shp, gpkg, sql, ...) avec la méthode `GeoDataFrame.to_file()`.
NB: Certains formats (le shapefile par exemple) gèrent très mal les noms de champs trop longs.

Bien sûr, on peux également recharger en mémoire un fichier existant avec geopandas :

In [ ]:
gdf_velov = gpd.read_file('datasets/output/stations.gpkg')
gdf_velov.head()